# Encoder Architecture from Scratch

## Class Building

In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset

import numpy as np
import matplotlib.pyplot as plt

In [2]:
class MutiHeadAttention(nn.Module):
    def __init__(self, d_k, d_model, n_heads):
        super().__init__()

        # assumiamo d_v = d_k = d_q
        self.d_k = d_k
        self.n_heads = n_heads

        # Siccome python fa schifo con i for loops, creiamo automaticamente delle matrici con le heads concatenate
        # Stiamo cercando di implementare le operazioni matriciali utilizzando dei layer Linear. Questi tuttavia aggiungono un vettore bias
        # E' possibile settare a 0 il bias, volendo, ma non lo faremo
        self.query = nn.Linear(d_model, d_k * n_heads)
        self.key = nn.Linear(d_model, d_k * n_heads)
        self.value = nn.Linear(d_model, d_k * n_heads)

        # Layer finale fully connected
        self.fc = nn.Linear(d_k * n_heads, d_model)


    def forward(self, k, q, v, mask=None):
        """
        In input, qui, abbiamo i valori di key, query e value che abbiamo in input.
        La formula, infatti, richiede dei vettori di key, query e value che, moltiplicati per alcune matrici di pesi, restituiscono
        dei vettori con un "significato". Qui, in poche parole, stiamo creando le quey, le key e i valori che comprendono il testo
        """
        # Teniamo traccia delle dimensioni: l'input è formato da N esempi di lunghezza T di vettori di dimensione d_model
        q = self.query(q) # N x T x (hd_k) -> h numero di attention heads
        k = self.key(k) # N x T x (hd_k)
        v = self.value(v) # N x T x (hd_v)

        N = q.shape[0]
        T = q.shape[1]

        """
        Come se non bastasse, nonostante abbiamo un tensore tridimensionale, per far funzionare le cose dobbiamo fare uno split ulteriore
        Vogliamo portare la dimensione a N x T x h x d_k, per poi invertire la dimensione 1 con la dimensione 2
        Per far questo, usiamo due funzioni di torch: view, che è l'equivalente di reshape, e transpose. Una trasposizione a più dimensioni
        ti permette di invertire due dimensioni passandoci in input gli assi
        """
        q = q.view(N, T, self.n_heads, self.d_k).transpose(1, 2)
        k = k.view(N, T, self.n_heads, self.d_k).transpose(1, 2)
        v = v.view(N, T, self.n_heads, self.d_k).transpose(1, 2)

        """
        Facciamo il primo passaggio: Q*K.T / sqrt(d_k)
        (N x h x T x d_k) * (N x h x d_k x T) = (N x h x T x T) -> vogliamo una matrice T x T per gli attention head, per ogni head, per ogni batch
        la @ in torch indica un prodotto scalare, utile per il broadcasting in questo caso. Il broadcast implementa implicitamente un for loop
        In poche parole le prime due dimensioni vengono calcolate "element-wise" mentre le ultime due con un dot product. come se facessimo
        for n in range(N): for h in range(H): score[n, h] = q[n, h] dot k[n, h].transpose
        """
        attn_scores = q @ k.transpose(-2, -1) / math.sqrt(self.d_k)
        # Applichiamo masking
        # Il masking serve per riconoscere quali dei token è token e quale è padding ed ha dimensione (N x T), quindi bidimensionale
        # Noi dobbiamo applicarla ad un tensore quadridimensionale. Usando mask [:, None, None, :] stiamo aggiungendo dimensioni extra, tanto da
        # avere come risultato qualcosa di simile a (N x 1 x 1 x T)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(
                mask[:, None, None, :] == 0, float("-inf")
            )
        # applichiamo softmax all'ultima dimensione
        attn_weights = F.softmax(attn_scores, dim=-1)

        # ora moltiplichiamo i pesi dell'attenzione per il valore
        # (N x h x T x T) x (N x h x T x d_k) = (N x h x T x d_k)
        A = attn_weights @ v

        # facciamo tornare tutto alla dimensione iniziale prima del layer finale
        # L'uso di contiguous serve a essere sicuri che la matrice sia correttamente piazzata in memoria (contiguità)
        A = A.transpose(1, 2) # (N x T x h x d_k)
        A = A.contiguous().view(N, T, self.n_heads * self.d_k)

        # ritorniamo la proiezione lineare
        return self.fc(A)
        

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, d_k, d_model, n_heads, dropout_prob=0.1):
        super().__init__()

        self.ln1 = nn.LayerNorm(d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.mha = MutiHeadAttention(d_k, d_model, n_heads)
        # Ricorda: in torch un layer si istanzia esplicitando input e output, ad eccezione della batch ovviamente)
        self.ann = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.GELU(),
            nn.Linear(d_model * 4, d_model),
            nn.Dropout(p=dropout_prob)
        )
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, x, mask=None):
        # input (N x T x d_model), mask = (N x T)
        x = self.ln1(x + self.mha(x, x, x, mask))
        x = self.ln2(x + self.ann(x))
        x = self.dropout(x)
        return x

#### La formula in breve:
La formula del positional encoding è la seguente:

$PE(pos, 2i) = sin(\frac{pos}{10000^{\frac{2i}{dmodel}}})$

$PE(pos, 2i+1) = cos(\frac{pos}{10000^{\frac{2i}{dmodel}}})$

Esiste un modo semplice per riscrivere il tutto nell'operazione giù per evitare caos con la stabilità numerica, dato che 10000 è un numero molto grande mentre e è molto piccolo.

Se prendiamo $10000^{-\frac{2i}{dmodel}}$ possiamo trasformare il tutto con: $(e^{-log(10000)})^{\frac{2i}{dmodel}}$ = $(e^{\frac{-log(10000)*2i}{dmodel}})$

In [4]:
"""
Ricordiamo le formule per il positional encoding:
PE(pos, 2i) = sin(pos/10000^[2i/dmodel])
PE(pos, 2i+1) = cos(pos/10000^[2i/dmodel])
"""
# Max_len serve perchè dobremmo sapere quanti output dobbiamo precalcolare
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2048, dropout_prob=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_prob)

        # creiamo un tensore di interi tra 0 e mex_len, ci aggiungiamo una dimensione. L'uso di "1" ci fa creare un tensore colonna
        # indicando l'asse che vogliamo espandere. Otteniamo un array (max_len x 1)
        position = torch.arange(max_len).unsqueeze(1) # Pos in formula
        exp_term = torch.arange(0, d_model, 2) # Array di esponenti, da 0 a d_model contando a due a due. Sono gli esponenti di 10k
        div_term = torch.exp(exp_term * (- math.log(10000) / d_model)) # VEDERE SOPRA

        # inzializziamo pe di dimensione (1 x max_len x d_model). La prima dimensione serve per rendere tutto broadcastabile
        # in quanto pe dev'essere aggiunto all'input che ha dimensione N x T x d_model
        pe = torch.zeros(1, max_len, d_model)
        # assegnamo i valori corretti di pe. Nota che usiamo pos * div_term perchè div term ha l'esponenente negativo
        pe[0,:,0::2] = torch.sin(position * div_term) # ultima dimensione da 0 ogni 2 (pari)
        pe[0,:,1::2] = torch.cos(position * div_term) # ultima dimensione: da 1 ogni 2 (dispari)

        # Questa istruzione ci permette di salvare e caricare correttamente il modello
        self.register_buffer("pe", pe)

    def forward(self, x):
        # x = input di shape (N x T x d_model)
        # Ricordiamo: pytorch ci permette di mettere in input sequenze di dimensioni differenti, quindi T variabile
        x = x + self.pe[:, :x.size(1), :] # Qui stiamo dicendo di elaborare fino alla lunghezza massima sulla seconda dim.
        return self.dropout(x)
        

In [5]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, max_len, d_k, d_model, n_heads, n_layers, n_classes, dropout_prob):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)
        transformer_blocks = [TransformerBlock(d_k, d_model, n_heads, dropout_prob) for _ in range(n_layers)]
        self.transformers_blocks = nn.Sequential(*transformer_blocks)
        self.ln = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model, n_classes)

    def forward(self, x, mask=None):
        x = self.embedding(x)
        x = self.pos_encoding(x)
        # facciamo il for, in quanto è necessario passare la mask per ogni transformer block
        for block in self.transformers_blocks:
            x = block(x, mask)

        # Qui facciamo una cosa: siccome stiamo codificando un'informazione, prendiamo uno degli hidden vectors a caso (il primo per convenzione)
        # Shape partenza: (N x T x d_model) -> (N x d_model)
        x = x[:, 0, :]
        x = self.ln(x)
        x = self.fc(x)
        return x

### Testing

In [6]:
# Testiamo il funzionamento dell'encoder!
model = Encoder(20000, 1024, 16, 64, 4, 2, 5, 0.1)

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

cuda:0


Encoder(
  (embedding): Embedding(20000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformers_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): MutiHeadAttention(
        (query): Linear(in_features=64, out_features=64, bias=True)
        (key): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=256, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, 

In [8]:
# Creiamo un vettore di dimensione 8 x 512 con valori interi che vanno da 0 alla vocab_size dummy inserita
# La dimensione specificata è NxT
x = np.random.randint(0, 20000, size=(8,512))
x_t = torch.tensor(x).to(device)

In [9]:
# Creiamo una mask corrispondente di dimensione
mask = np.ones((8, 512))
mask[:, 256:] = 0
mask_t = torch.tensor(mask).to(device)

In [10]:
y = model(x_t, mask_t)

In [11]:
# la shape ha senso. Sono 8 samples con le 5 classi corrispondenti
y.shape

torch.Size([8, 5])

## Tokenizer & Data Collator

In [12]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
checkpoint = "distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

C:\Users\emili\Desktop\transformers_course\transformers-env\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [14]:
# Teniamo le cose semplici ed usiamo il dataset della sentiment analysis per il GLUE benchmark
raw_datasets = load_dataset("glue", "sst2")

In [15]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [16]:
def tokenize_fn(batch):
    return tokenizer(batch["sentence"], truncation=True)

In [17]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

Map: 100%|███████████████████████████████████████████████████████████████| 1821/1821 [00:00<00:00, 28504.99 examples/s]


In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [20]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [21]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [22]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

## Training

### Data Loading

In [23]:
from torch.utils.data import DataLoader

In [24]:
train_loader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator # Inseriamo il data collator.
)

In [25]:
# Valid non ha lo shuffling
valid_loader = DataLoader(
    tokenized_datasets["validation"],
    batch_size=32,
    collate_fn=data_collator
)

In [26]:
# Vediamo come funziona:
for batch in train_loader:
    for k, v in batch.items():
        print("k:", k, "v.shape:", v.shape)
    break

"""
Esempio output da analizzare:
k: labels v.shape: torch.Size([32]) ---> 32 è la batch size, sono le label perchè abbiamo usato 32 di batch
k: input_ids v.shape: torch.Size([32, 51]) ----> 51, invece, è la dimensione T. In questo caso abbiamo una sequenza di 51 con il collator
k: attention_mask v.shape: torch.Size([32, 51]) ---> La mask si porta appresso la stessa dim
"""; 

k: labels v.shape: torch.Size([32])
k: input_ids v.shape: torch.Size([32, 34])
k: attention_mask v.shape: torch.Size([32, 34])


In [27]:
# Vediamo quali sono i valori delle labels (num_classes)
set(tokenized_datasets["train"]["labels"])

{0, 1}

In [28]:
# Check per la vocabulary size:
tokenizer.vocab_size

28996

In [29]:
# Check max sequence lenght per il tokenizer:
tokenizer.model_max_length

512

### Model Building

In [30]:
# Ricordiamo: ci sono alcune cose che vengono da costruzione del tokenizer, altre scelte arbitrariamente
model = Encoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.model_max_length,
    d_k=16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=2, # Numero delle label che abbiamo nel dataset
    dropout_prob=0.1
)
model.to(device);

### Loss & Optimizer

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

### Train Function

In [32]:
from datetime import datetime

In [33]:
def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
    train_losses = np.zeros(epochs)
    valid_losses = np.zeros(epochs)

    for it in range(epochs):
        # SETTIAMO IN TRAIN MODE
        model.train()
        t0 = datetime.now()
        train_loss = 0
        n_train = 0 # numero di campioni di training

        for batch in train_loader:
            # muoviamo su GPU i dati
            batch = {k: v.to(device) for k, v in batch.items()}

            # azzeriamo i gradienti (evita di accumulare il gradiente da iterazioni precedenti)
            optimizer.zero_grad()

            # forward pass + loss
            outputs = model(batch["input_ids"], batch["attention_mask"])
            loss = criterion(outputs, batch["labels"])

            # backpropagation
            loss.backward()
            optimizer.step()

            # train loss computing:
            # la loss function di pytorch è una media su tutti i campioni di un batch.
            # per ottenere la loss su tutto il train set nella epoch, dobbiamo moltiplicare per il numero di campioni in un batch e sommare su tutto
            # infine teniamo traccia di quanti campioni abbiamo per dividere
            bs = batch["input_ids"].size(0)
            train_loss += loss.item() * bs
            n_train += bs

        train_loss = train_loss / n_train

        # SWITCH A VALIDATION MODE (non facciamo step di azzeramento gradienti e backpropagation, solo forward)
        model.eval()
        valid_loss = 0
        n_valid = 0

        for batch in valid_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(batch["input_ids"], batch["attention_mask"])
            loss = criterion(outputs, batch["labels"])
            bs = batch["input_ids"].size(0)
            valid_loss += loss.item() * bs
            n_valid += bs
        valid_loss = valid_loss / n_valid

        train_losses[it] = train_loss
        valid_losses[it] = valid_loss

        dt = datetime.now() - t0
        print(f"Epoch: {it+1}/{epochs} --- Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}, Duration: {dt}")

    return train_losses, valid_losses

In [34]:
train_losses, valid_losses = train(model, criterion, optimizer, train_loader, valid_loader, epochs=4)

Epoch: 1/4 --- Train Loss: 0.5341, Valid Loss: 0.5090, Duration: 0:00:17.210935
Epoch: 2/4 --- Train Loss: 0.3684, Valid Loss: 0.4825, Duration: 0:00:16.586278
Epoch: 3/4 --- Train Loss: 0.2991, Valid Loss: 0.4938, Duration: 0:00:16.416945
Epoch: 4/4 --- Train Loss: 0.2606, Valid Loss: 0.5089, Duration: 0:00:17.159260


### Testiamo l'accuracy del modello

In [35]:
from sklearn.metrics import f1_score, roc_auc_score

In [36]:
model.eval()
n_correct = 0
n_total = 0
f1 = 0
for batch in train_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(batch["input_ids"], batch["attention_mask"])
    _, predictions = torch.max(outputs, 1) # torch.max ci torna il valore e l'indice (che ci serve). 1 è la dimensione sul quale calcoliamo il max
    partial_f1 = f1_score(batch["labels"].cpu().numpy(), predictions.cpu().numpy())
    n_correct += (predictions == batch["labels"]).sum().item()
    n_total += batch["labels"].shape[0]
    f1 += partial_f1 * batch["labels"].shape[0]

train_acc = n_correct / n_total
f1 = f1 / n_total
print("Training Accuracy:", train_acc, "f1:", f1)

n_correct = 0
n_total = 0
f1 = 0
for batch in valid_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(batch["input_ids"], batch["attention_mask"])
    _, predictions = torch.max(outputs, 1) # torch.max ci torna il valore e l'indice (che ci serve). 1 è la dimensione sul quale calcoliamo il max
    n_correct += (predictions == batch["labels"]).sum().item()
    partial_f1 = f1_score(batch["labels"].cpu().numpy(), predictions.cpu().numpy())
    n_total += batch["labels"].shape[0]
    f1 += partial_f1 * batch["labels"].shape[0]

valid_acc = n_correct / n_total
f1 = f1 / n_total
print("Validation Accuracy:", valid_acc,  "f1:", f1)

Training Accuracy: 0.9313575554202735 f1: 0.9374092437783264
Validation Accuracy: 0.786697247706422 f1: 0.7917288733166009
